In [72]:
import pandas as pd
import numpy as np

def vcamsLGA(factor, year, sheet = 0):
    path = './raw_data/vcamsLGA/VCAMS_'+factor+'.xlsx'
    df = pd.read_excel(path, sheet_name=sheet)
    # remove unnamed rows
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    # remove aggregates (victoria)
    df = df[~df['LGA'].str.contains('Victoria')]
    df = df[df['Year'] == year]
    df['LGA'] = df['LGA'].replace('\([a-zA-Z]*\)','', regex = True)
    df['LGA'] = df['LGA'].str.strip()
    df = df.set_index('LGA')
    df.loc[df['Indicator'] == 'NDP'] = np.nan
    df['Indicator'] = pd.to_numeric(df['Indicator'])
    df = df.drop(['Numerator', 'Denominator', 'Year'], axis =1).rename({'Indicator': factor}, axis = 1)
    return df

def vcamsDHS(name, year=2014, sheet = 0):
    path = './raw_data/vcamsDHS/VCAMS_'+name+'.xlsx'
    df = pd.read_excel(path, sheet_name=sheet)
    df = df[df['Year'] == year]
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    df = df[~df['DHS AREA'].str.contains('Victoria')]
    df['DHS AREA'] = df['DHS AREA'].replace('Area', '', regex=True)
    df['DHS AREA'] = df['DHS AREA'].str.strip()
    df = df.drop(['RSE', 'Year'], axis=1)
    df = df.set_index('DHS AREA')
    df.loc[df['Indicator'] == 'NDP'] = np.nan
    df['Indicator'] = pd.to_numeric(df['Indicator'], errors='ignore')
    df = df.rename(columns={'Indicator': name})
    return df



In [85]:
depressionDHS = pd.read_csv('./wrangled/depression2018DHS.csv')
depressionDHS['DHS AREA'] = depressionDHS['DHS AREA'].replace('Area', '', regex=True)
depressionDHS['DHS AREA'] = depressionDHS['DHS AREA'].str.strip()
depressionDHS = depressionDHS.set_index('DHS AREA')

In [18]:
vcamsDHS('financial')

,Indicator
DHS AREA,
Barwon Area,0.047090
Bayside Peninsula Area,0.045726
Brimbank Melton,0.040905
Central Highlands Area,0.020097
Goulburn Area,0.047842
Hume Moreland Area,0.022939
Inner Eastern Melbourne Area,0.019938
Inner Gippsland Area,0.067882
Loddon Area,NaN


In [43]:
names = [s.strip('\n') for s in open('./raw_data/vcamsDHS/names.txt', "r").readlines()

SyntaxError: invalid syntax (<ipython-input-43-37de3441114d>, line 1)

In [57]:
files = {k: int(v) for line in open('./raw_data/vcamsDHS/names.txt', "r") for (k,v) in [line.split()]}

In [90]:
vcams_DHS_data = [vcamsDHS(i, year = files[i]) for i in files.keys()]

In [91]:
DHS_data = depressionDHS.join(vcams_DHS_data)

In [92]:
DHS_data.to_csv('./wrangled/DHSdata.csv')